In [19]:
all_text = [['I', 'enjoy', 'flying'], ['I', 'like', 'NLP'], ['I', 'like', 'deep', 'learning'], ['learning', 'deep']]

In [20]:
vocabulary = set(['<UNK>'])
word_count = 0

for text in all_text:
  vocabulary |= set(text)
  word_count += len(text)
    
print('|D|', len(all_text))
print('|V|', len(vocabulary))
print('|W|', word_count)


|D| 4
|V| 8
|W| 12


In [21]:
from tqdm.notebook import tqdm
counts = {}
for text in tqdm(all_text):
  for word in text:
    counts[word] = counts.get(word,0) + 1

# for word in counts:
#   if counts[word] < 4:
#     vocabulary.remove(word)
print(len(vocabulary))

  0%|          | 0/4 [00:00<?, ?it/s]

8


In [22]:
vocab2index = {v:i for i,v in enumerate(vocabulary)}
index2vocab = {i:v for i,v in enumerate(vocabulary)}

In [23]:
print(vocab2index)
print(index2vocab)

{'I': 0, '<UNK>': 1, 'deep': 2, 'like': 3, 'enjoy': 4, 'NLP': 5, 'flying': 6, 'learning': 7}
{0: 'I', 1: '<UNK>', 2: 'deep', 3: 'like', 4: 'enjoy', 5: 'NLP', 6: 'flying', 7: 'learning'}


In [24]:
def get_cooccurrences(texts,window):
  cooccurrences = {}
  for text in texts:
    for i in range(len(text)):
      for j in range(1, window + 1):
        if(i+j < len(text)):
          cooccurrences[(text[i], text[i+j])] =  cooccurrences.get((text[i], text[i+j]), 0) + 1
  return cooccurrences

In [25]:
cooccurrences = get_cooccurrences(all_text,1)

In [26]:
cooccurrences

{('I', 'enjoy'): 1,
 ('enjoy', 'flying'): 1,
 ('I', 'like'): 2,
 ('like', 'NLP'): 1,
 ('like', 'deep'): 1,
 ('deep', 'learning'): 1,
 ('learning', 'deep'): 1}

In [27]:
data = []
rows = []
cols = []
keys_list = list(cooccurrences.keys())
keys_set = set()
for keys in keys_list:
  (word1, word2) = keys
  if(keys not in keys_set):
    reversed_keys = (word2, word1) 
    if(reversed_keys not in keys_set):
      rows.append(vocab2index[word1])
      cols.append(vocab2index[word2])
      data.append(cooccurrences.get(keys) + cooccurrences.get(reversed_keys, 0))
      rows.append(vocab2index[word2])
      cols.append(vocab2index[word1])
      data.append(cooccurrences.get(keys) + cooccurrences.get(reversed_keys, 0))
      keys_set.add(keys)
      keys_set.add(reversed_keys)


    

In [28]:
for i in range(len(rows)):
  print(f'{index2vocab[rows[i]]} {index2vocab[cols[i]]} {data[i]}')

I enjoy 1
enjoy I 1
enjoy flying 1
flying enjoy 1
I like 2
like I 2
like NLP 1
NLP like 1
like deep 1
deep like 1
deep learning 2
learning deep 2


In [29]:
import scipy
cooccurrences = scipy.sparse.coo_matrix((data,(rows,cols)),shape=(len(vocab2index),len(vocab2index)))

In [30]:
cooccurrences_array = cooccurrences.toarray()
cooccurrences_array

array([[0, 0, 0, 2, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2],
       [2, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0]])

In [16]:
from scipy.stats import ortho_group
from scipy.sparse import csc_matrix, diags
from scipy.sparse.linalg import svds
#(cooccurrences:scipy.sparse.coo_matrix)->dict(str,np.array)
def get_svd_word_vectors(cooccurrences):
  word_vectors = {}
  cooccurrences = cooccurrences.asfptype()
  u2, s2, vT2 = svds(cooccurrences, k=3)
  print(u2)
  print(s2)
  print(vT2)

  return word_vectors

svd_vecs = get_svd_word_vectors(cooccurrences)

[[ 4.07127648e-01  6.41170943e-01 -3.07883784e-01]
 [-6.47885395e-18  2.09004732e-17 -7.83207624e-20]
 [-5.48521597e-01  5.63836264e-01 -2.70748456e-01]
 [ 2.04959147e-01 -3.65355242e-01 -7.60855808e-01]
 [ 3.14015631e-01 -1.26886544e-01 -2.64242449e-01]
 [ 1.15265403e-01  2.73152958e-01 -1.31165280e-01]
 [ 1.76596843e-01  9.48650268e-02 -4.55532237e-02]
 [-5.90156343e-01 -1.94401464e-01 -4.04842920e-01]]
[1.81807791 2.78545696 2.78545696]
[[ 0.39818641  0.         -0.53647511  0.20956148  0.32106682  0.11273397
   0.17271847 -0.60340824]
 [-0.30788378  0.         -0.27074846  0.76085581  0.26424245 -0.13116528
  -0.04555322  0.40484292]
 [-0.64117094  0.         -0.56383626 -0.36535524 -0.12688654 -0.27315296
  -0.09486503 -0.19440146]]
